In [1]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from elastic_rods import EnergyType
from bending_validation import suppress_stdout as so
import numpy.linalg as la

In [2]:
import numpy as np

In [3]:
input_path = '../../data/sphere_cap_0.3.json'
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
import mesh
target_mesh = mesh.Mesh(input_data['target_v'], input_data['target_f'])

### Initialization

In [4]:
curr_um = umbrella_mesh.UmbrellaMesh(io)
thickness = io.material_params[6]
curr_um.targetDeploymentHeight = thickness

In [5]:
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
view.averagedMaterialFrames = True
# view.setCameraParams(camParam)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.7066666666666668, children=(PointLight(color='#999999', position=(0…

### Parameters

In [6]:
curr_um.uniformDeploymentEnergyWeight = 0.01
curr_um.targetDeploymentHeight = thickness * 1
curr_um.attractionWeight = 0.01

In [7]:
perturbation = np.random.uniform(-1e-3, 1e-3, curr_um.numDoF())
dof = curr_um.getDoFs()
curr_um.setDoFs(dof + perturbation)
view.update()

In [8]:
curr_um.updateSourceFrame()

In [9]:
curr_um.energy(umbrella_mesh.UmbrellaEnergyType.Elastic), curr_um.energy(umbrella_mesh.UmbrellaEnergyType.Deployment), curr_um.energy(umbrella_mesh.UmbrellaEnergyType.Attraction), curr_um.energy()

(1.2950467575967137,
 -0.1716742409542453,
 9.834387701608016e-05,
 1.1234708605194843)

### Validate Joint Hessian Vector Product

In [10]:
jIndex = curr_um.segment(31).startJoint

In [11]:
evalHessianJS = curr_um.getTerminalEdgeSensitivity(31, umbrella_mesh.TerminalEdge.Start, True, True)

In [12]:
total_perturbation = np.random.uniform(0, 1e-3, curr_um.numDoF())
joint_perturbation = np.zeros(5)
joint_perturbation[:4] = total_perturbation[curr_um.dofOffsetForJoint(jIndex) + 3: curr_um.dofOffsetForJoint(jIndex) + 3 + 4]
joint_perturbation[4] = total_perturbation[curr_um.dofOffsetForJoint(jIndex) + 3 + 4 + evalHessianJS.localSegmentIndex]

In [13]:
deltaJS = curr_um.getTerminalEdgeSensitivity(31, umbrella_mesh.TerminalEdge.Start, True, total_perturbation)

In [14]:
error = np.array(evalHessianJS.hessian) @ joint_perturbation - deltaJS.delta_jacobian

In [15]:
la.norm(error)

1.3559149445364337e-20

### Hessian Vector Product

In [16]:
energyType = umbrella_mesh.UmbrellaEnergyType.Elastic

In [17]:
# Get Hessian
variableDesignParameters = True
hessian = curr_um.hessian(variableDesignParameters = variableDesignParameters, umbrellaEnergyType = energyType)
hessian.reflectUpperTriangle()
hessian = hessian.compressedColumn()

In [18]:
n_dof = curr_um.numExtendedDoF() if variableDesignParameters else curr_um.numDoF()
print("Num of DOF: ", n_dof)
perturb = np.random.uniform(0, 1e-3, n_dof)
# perturb = np.ones(n_dof)

input_vector = perturb
code_output = curr_um.applyHessian(input_vector, variableDesignParameters, umbrellaEnergyType = energyType)
matrix_output = hessian * input_vector
error = la.norm(code_output - matrix_output) / (la.norm(code_output) + 1e-8) 
print(error)

Num of DOF:  8531
8.975144657232476e-16


In [19]:
# offsets
jv_offset = curr_um.dofOffsetForJoint(0)
rl_offset = curr_um.restLenDofOffsetForSegment(0)

In [20]:
var_types = ['FULL', 'Rod', 'Joint', 'Design']
var_indices = {'FULL': range(0, n_dof),
                'Rod': range(0, jv_offset),
                'Joint' : range(jv_offset, rl_offset),
                'Design': range(rl_offset, n_dof)}

### Validate Hessian Vector Product Per Block

In [21]:
for i, vi in enumerate(var_types[1:]):
    for j, vj in enumerate(var_types[1:]):
        input_vector = np.zeros_like(perturb)
        input_vector[var_indices[vj]] = perturb[var_indices[vj]]
        code_output = curr_um.applyHessian(input_vector, variableDesignParameters, umbrellaEnergyType = energyType)
        matrix_output = hessian * input_vector
        code_output = code_output[var_indices[vi]]
        matrix_output = matrix_output[var_indices[vi]]
        error = la.norm(code_output - matrix_output) / (la.norm(code_output) + 1e-8)
        print('Error for {} {} block: {}, {}, {}'.format(vi, vj, error, la.norm(code_output), la.norm(matrix_output)))

Error for Rod Rod block: 8.448815704849058e-16, 63.33364496764193, 63.33364496764194
Error for Rod Joint block: 9.358219765461164e-16, 27.036441020819336, 27.036441020819336
Error for Rod Design block: 3.9066737401923396e-16, 0.32406311240831526, 0.3240631124083153
Error for Joint Rod block: 8.425231143381754e-16, 24.029323919942687, 24.029323919942687
Error for Joint Joint block: 7.883483830029137e-16, 27.21149580854279, 27.21149580854279
Error for Joint Design block: 3.074218306670075e-16, 0.2375078463805892, 0.2375078463805892
Error for Design Rod block: 3.186836558594981e-16, 0.3698211611441727, 0.3698211611441727
Error for Design Joint block: 2.6880177672903926e-16, 0.33706916842706636, 0.33706916842706636
Error for Design Design block: 2.7315365070897606e-16, 0.5216426954422501, 0.5216426954422501


## Per Arm Rest Length

In [22]:
# Get Hessian
hessian = curr_um.hessianPerArmRestlen(umbrellaEnergyType = energyType)
hessian.reflectUpperTriangle()
hessian = hessian.compressedColumn()

In [23]:
n_dof = curr_um.numExtendedDoFPARL()
print("Num of DOF: ", n_dof)
perturb = np.random.uniform(0, 1e-3, n_dof)

input_vector = perturb
code_output = curr_um.applyHessianPerArmRestlen(input_vector, umbrellaEnergyType = energyType)
matrix_output = hessian * input_vector
error = la.norm(code_output - matrix_output) / la.norm(code_output)
print(error)

Num of DOF:  5171
1.935389047535645e-15


In [24]:
# offsets
jv_offset = curr_um.dofOffsetForJoint(0)
rl_offset = curr_um.restLenDofOffsetForSegment(0)

print(jv_offset, rl_offset)

4002 5111


In [25]:
var_types = ['FULL', 'Rod', 'Joint', 'Design']
var_indices = {'FULL': range(0, n_dof),
                'Rod': range(0, jv_offset),
                'Joint' : range(jv_offset, rl_offset),
                'Design': range(rl_offset, n_dof)}

### Validate Hessian Vector Product Per Block

In [26]:
for i, vi in enumerate(var_types[1:]):
    for j, vj in enumerate(var_types[1:]):
        input_vector = np.zeros_like(perturb)
        input_vector[var_indices[vj]] = perturb[var_indices[vj]]
        code_output = curr_um.applyHessianPerArmRestlen(input_vector, umbrellaEnergyType = energyType)
        matrix_output = hessian * input_vector
        code_output = code_output[var_indices[vi]]
        matrix_output = matrix_output[var_indices[vi]]
        error = la.norm(code_output - matrix_output) / la.norm(code_output)
        print('Error for {} {} block: {}, {}, {}'.format(vi, vj, error, la.norm(code_output), la.norm(matrix_output)))

Error for Rod Rod block: 1.8569484479737195e-15, 87.17146872344424, 87.17146872344422
Error for Rod Joint block: 2.3916729623481363e-15, 33.26086163030682, 33.26086163030681
Error for Rod Design block: 2.4964554483488936e-15, 0.005115230772368281, 0.005115230772368281
Error for Joint Rod block: 1.7123827773930446e-15, 31.79886916466375, 31.798869164663742
Error for Joint Joint block: 1.8431157559036116e-15, 30.964031238571895, 30.964031238571888
Error for Joint Design block: 4.074459932466936e-16, 0.020449068674004076, 0.020449068674004076
Error for Design Rod block: 3.434573138312989e-15, 0.003208091814437906, 0.0032080918144379057
Error for Design Joint block: 4.79014208214656e-16, 0.019842913816744693, 0.019842913816744693
Error for Design Design block: 2.956622036788434e-16, 0.01775655686552309, 0.01775655686552309
